# Issues in thread synchronization

We might encounter issues while implementing concurrent programming or applying synchronizing primitives. In this section, we will discuss two major issues. The issues are:

- Deadlock
- Race condition


## Race condition

This is one of the major issues in concurrent programming. Concurrent access to shared resources can lead to race condition. A race condition may be defined as the occurring of a condition when two or more threads can access shared data and then try to change its value at the same time. Due to this, the values of variables may be unpredictable and vary depending on the timings of context switches of the processes.

### Example

Consider this example to understand the concept of race condition

**Step 1** − In this step, we need to import threading module −

In [1]:
import threading

**Step 2** − Now, define a global variable, say x, along with its value as 0 −

In [2]:
x = 0

**Step 3** − Now, we need to define the `increment_global()` function, which will do the increment by 1 in this global function x −

In [3]:
def increment_global():

   global x
   x += 1

**Step 4** − In this step, we will define the `taskofThread()` function, which will call the `increment_global()` function for a specified number of times; for our example it is 50000 times −

In [4]:
def taskofThread():

   for _ in range(500000):
      increment_global()

**Step 5** − Now, define the `main()` function in which threads t1 and t2 are created. Both will be started with the help of the `start()` function and wait until they finish their jobs with the help of `join()` function.



In [5]:
def main():
   global x
   x = 0
   
   t1 = threading.Thread(target= taskofThread)
   t2 = threading.Thread(target= taskofThread)

   t1.start()
   t2.start()

   t1.join()
   t2.join()

**Step 6** − Now, we need to give the range as in for how many iterations we want to call the `main()` function. Here, we are calling it for 5 times.

In the output shown below, we can see the effect of race condition as the value of x after each iteration is expected 1000000. However, there is lots of variation in the value. This is due to the concurrent access of threads to the shared global variable x.

In [6]:
if __name__ == "__main__":
   for i in range(5):
      main()
      print("x = {1} after Iteration {0}".format(i,x))

x = 643217 after Iteration 0
x = 854237 after Iteration 1
x = 887859 after Iteration 2
x = 918151 after Iteration 3
x = 844815 after Iteration 4


### Dealing with race condition using locks

As we have seen the effect of race condition in the above program, we need a synchronization tool, which can deal with race condition between multiple threads. In Python, the `<threading>` module provides Lock class to deal with race condition. Further, the Lock class provides different methods with the help of which we can handle race condition between multiple threads. The methods are described below −

#### `acquire()` method

This method is used to acquire, i.e., blocking a lock. A lock can be blocking or non-blocking depending upon the following true or false value −

- **With value set to True** − If the `acquire()` method is invoked with True, which is the default argument, then the thread execution is blocked until the lock is unlocked.

- **With value set to False** − If the `acquire()` method is invoked with False, which is not the default argument, then the thread execution is not blocked until it is set to true, i.e., until it is locked.

#### `release()` method

This method is used to release a lock. Following are a few important tasks related to this method −

- If a lock is locked, then the `release()` method would unlock it. Its job is to allow exactly one thread to proceed if more than one threads are blocked and waiting for the lock to become unlocked.

- It will raise a `ThreadError` if lock is already unlocked.

Now, we can rewrite the above program with the lock class and its methods to avoid the race condition. We need to define the `taskofThread()` method with lock argument and then need to use the `acquire()` and `release()` methods for blocking and non-blocking of locks to avoid race condition.

#### Example

Following is example of python program to understand the concept of locks for dealing with race condition −

In [7]:
import threading

x = 0

def increment_global():

   global x
   x += 1

def taskofThread(lock):

   for _ in range(500000):
      lock.acquire()
      increment_global()
      lock.release()

def main():
   global x
   x = 0

   lock = threading.Lock()
   t1 = threading.Thread(target = taskofThread, args = (lock,))
   t2 = threading.Thread(target = taskofThread, args = (lock,))

   t1.start()
   t2.start()

   t1.join()
   t2.join()

if __name__ == "__main__":
   for i in range(5):
      main()
      print("x = {1} after Iteration {0}".format(i,x))

x = 1000000 after Iteration 0
x = 1000000 after Iteration 1
x = 1000000 after Iteration 2
x = 1000000 after Iteration 3
x = 1000000 after Iteration 4


The output shows that the effect of race condition is neglected; as the value of x, after each & every iteration, is now 100000, which is as per the expectation of this program.



## Deadlocks − The Dining Philosophers problem

Deadlock is a troublesome issue one can face while designing the concurrent systems. We can illustrate this issue with the help of the dining philosopher problem as follows −

Edsger Dijkstra originally introduced the dining philosopher problem, one of the famous illustrations of one of the biggest problem of concurrent system called deadlock.

In this problem, there are five famous philosophers sitting at a round table eating some food from their bowls. There are five forks that can be used by the five philosophers to eat their food. However, the philosophers decide to use two forks at the same time to eat their food.

Now, there are two main conditions for the philosophers. First, each of the philosophers can be either in eating or in thinking state and second, they must first obtain both the forks, i.e., left and right. The issue arises when each of the five philosophers manages to pick the left fork at the same time. Now they all are waiting for the right fork to be free but they will never relinquish their fork until they have eaten their food and the right fork would never be available. Hence, there would be a deadlock state at the dinner table.

### Deadlock in concurrent system

Now if we see, the same issue can arise in our concurrent systems too. The forks in the above example would be the system resources and each philosopher can represent the process, which is competing to get the resources.

### Solution with Python program

The solution of this problem can be found by splitting the philosophers into two types – **greedy philosophers** and **generous philosophers**. Mainly a greedy philosopher will try to pick up the left fork and wait until it is there. He will then wait for the right fork to be there, pick it up, eat and then put it down. On the other hand, a generous philosopher will try to pick up the left fork and if it is not there, he will wait and try again after some time. If they get the left fork then they will try to get the right one. If they will get the right fork too then they will eat and release both the forks. However, if they will not get the right fork then they will release the left fork.

### Example

The following Python program will help us find a solution to the dining philosopher problem.

The program uses the concept of greedy and generous philosophers. The program has also used the `acquire()` and `release()` methods of the `Lock` class of the `<threading>` module. We can see the solution in the following output −

In [8]:
import threading
import random
import time

class DiningPhilosopher(threading.Thread):

    running = True

    def __init__(self, xname, Leftfork, Rightfork):
        threading.Thread.__init__(self)
        self.name = xname
        self.Leftfork = Leftfork
        self.Rightfork = Rightfork

    def run(self):
        while(self.running):
            time.sleep( random.uniform(3,13))
            print ('%s is hungry.' % self.name)
            self.dine()

    def dine(self):
        fork1, fork2 = self.Leftfork, self.Rightfork

        while self.running:
            fork1.acquire(True)
            locked = fork2.acquire(False)
            if locked: break
            fork1.release()
            print ('%s swaps forks' % self.name)
            fork1, fork2 = fork2, fork1
        else:
            return

        self.dining()
        fork2.release()
        fork1.release()

    def dining(self):
        print ('%s starts eating '% self.name)
        time.sleep(random.uniform(1,10))
        print ('%s finishes eating and now thinking.' % self.name)

def Dining_Philosophers():
    forks = [threading.Lock() for n in range(5)]
    philosopherNames = ('1st','2nd','3rd','4th', '5th')

    philosophers= [DiningPhilosopher(philosopherNames[i], forks[i%5], forks[(i+1)%5]) \
       for i in range(5)]

    random.seed()
    DiningPhilosopher.running = True
    for p in philosophers: p.start()
    time.sleep(30)
    DiningPhilosopher.running = False
    print (" It is finishing.")

Dining_Philosophers()

3rd is hungry.
3rd starts eating 
5th is hungry.
5th starts eating 
2nd is hungry.
2nd swaps forks
4th is hungry.
1st is hungry.
3rd finishes eating and now thinking.
4th swaps forks
2nd starts eating 
5th finishes eating and now thinking.
1st swaps forks
4th starts eating 
3rd is hungry.
4th finishes eating and now thinking.
2nd finishes eating and now thinking.
1st starts eating 3rd starts eating 

3rd finishes eating and now thinking.
1st finishes eating and now thinking.
5th is hungry.
5th starts eating 
4th is hungry.
4th swaps forks
3rd is hungry.
3rd starts eating 
 It is finishing.
1st is hungry.
2nd is hungry.
3rd finishes eating and now thinking.
5th finishes eating and now thinking.
4th starts eating 
4th finishes eating and now thinking.
